In [2]:
#set chdir to current dir

import os
import sys

sys.path.insert(0,os.path.realpath(os.path.dirname(__file__)))
os.chdir(os.path.realpath(os.path.dirname(__file__)))

import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Output,Event,Input

import plotly
import plotly.graph_objs as go
import sqlite3
import pandas as pd

from collections import Counter
import string
import regex as re
from cache import cache
from config import stop_words
import time
import pickle

#it's ok to use one shared sqlite connection
#as we are making selects only, no need for any kind of serialization as well

conn=sqlite3.connect('Twitter_storage.db',check_same_thread=False)

punctuation=[str(i) for i in string.punctuation]

sentiment_colors={-1:"#EE6055",
                    -0.5:"#FDE74C",
                     0:"#FFE6AC",
                     0.5:"#D0F2DF",
                     1:"#9CEC5B",}

app_colors = {
    'background': '#0C0F0A',
    'text': '#FFFFFF',
    'sentiment-plot':'#41EAD4',
    'volume-bar':'#FBFC74',
    'someothercolor':'#FF206E',}

POS_NEG_NEUT=0.1

MAX_OF_LENGTH=100

app=dash.Dash(__name__)
app.layout=html.Div(
    [html.Div(className='container-fluid',children=[html.H2('Live Twitter Sentiment', style={'color':"#CECECE"}),
                                                                     html.H5('Search:', style={'color':app_colors['text']}),
                                                                    dcc.Input(id='sentiment_term',value='twitter',type='text',
                                                                             style={'color':app_colors['someothercolor']}),
                                                                    ],
                             style={'width':'98%','margin-left':10,'margin-right':10,'max-width':50000}),
                    
            html.Div(className='row',children=[html.Div(id='related-sentiment',children=html.Button('Loading related terms...',id='related_term_button'),className='col s12 m6 16',style={"word-wrap":"break-word"}),
                                               html.Div(id='recent-trending',className='col s12 m6 16',style={"word-wrap":"break-word"})]),
                     
            html.Div(className='row', children=[html.Div(dcc.Graph(id='live-graph', animate=False), className='col s12 m6 l6'),
                                            html.Div(dcc.Graph(id='historical-graph', animate=False), className='col s12 m6 l6')]),

            html.Div(className='row', children=[html.Div(id="recent-tweets-table", className='col s12 m6 l6'),
                                            html.Div(dcc.Graph(id='sentiment-pie', animate=False), className='col s12 m6 l6'),]),
                     
           
            dcc.Interval(
            id='graph-update',
            interval=1*1000
            ),
                     
            dcc.Interval(
            id='historical-update',
            interval=60*1000
            ),

            dcc.Interval(
            id='related-update',
            interval=30*1000
            ),

            dcc.Interval(
            id='recent-table-update',
            interval=2*1000
            ),

            dcc.Interval(
            id='sentiment-pie-update',
            interval=60*1000
            ), 
            
        ], style={'backgroundColor': app_colors['background'], 'margin-top':'-30px', 'height':'2000px',},
    
)


def df_resample_sizes(df,maxlen=MAX_DF_LENGTH):
    df_len=len(df)
    resample_amt=100
    vol_df=df.copy()
    vol_df['volume']=1
    
    ms_span=(df.index[-1]-df.index[0]).seconds*1000
    rs=int(ms_span/maxlen)
    
    df=df.resample('{}ms'.format(int(rs))).mean()
    df.dropna(inplace=True)
    
    vol_df=vol_df.resample('{}ms'.format(int(rs))).sum()
    vol_df.dropna(inplace=True)
    
    df=df.join(vol_df['volume'])
    
    return df

#make a counter with blacklist words and empty words with some big value -we'll use it later to filter counter
stop_words.append('')
blacklist_counter=Counter(dict(zip(stop_words,[1000000]*len(stop_words))))
    
#compile a regex for split operations (punctuation list,plus space and new line)

split_regex=re.compile("[\n"+re.escape("".join(punctuation))+']')

def related_sentiments(df,sentiment_term,how_many=15):
    try:
        related_words={}
        
        #its way faster to join strings to one string then use regex split using your punctuation list plus space and new line chars
        #regex precompiled above
        
        tokens=split_regex.split(' '.join(df['tweet'].values.tolist()).lower())
        
        #it is way faster to remove stop_words, sentiment_term and empty token by making another counter
        #with some big value and substracting (counter will substract and remove tokens with negative count)
        
        blacklist_counter_with_term=blacklist_counter.copy()
        blacklist_counter_with_term[sentiment_term]=1000000
        
        counts=(Counter(tokent)-blacklist_counter_with_term).most_common(15)
        
        for term,count in counts:
            try:
                df=pd.read_sql("SELECT sentiment.* FROM sentiment_fts fts LEFT JOIN sentiment ON fts.rowid=sentiment.id WHERE fts.sentiment_fts MATCH ? ORDER BY fts.rowid DESC LIMIT 200",conn,params=(term,))
                related_words[term]=[df['sentiment'].mean(),count]
                
            exception Exception as e:
                with open('errors.txt','a') as f:
                    f.write(str(e))
                    f.write('\n')
                    
        return related_words
    
    except Exception as e:
        with open('errors.txt','a') as f:
            f.write(str(e))
            f.write('\n')
            

def quick_color(s):
    #except return bg as app_colors['background']
    
    if s>=POS_NEG_NEUT:
        # positive
        return "#002C0D"
    
    elif s<=-POS_NEG_NEUT:
        #negative
        return "#270000"
    
    else:
        return app_colors['background']
    

def generate_table(df,max_rows=10):
    return html.Table(className="responsive-table",
                     children=[
                         html.Thread(
                             html.Tr(
                                 children=[
                                     html.Th(col.title()) for col in df.columns.values],
                                 style={'color':app_colors['text']}
                                 )
                         ),
                         html.Tbody(
                             [
                                 
                                 html.Tr(
                                 children=[
                                     html.Td(data) for data in d
                                 ],style={'color':app_colors['text'],
                                          'background-color':quick_color(d[2])}
                                 )
                                 
                            for d in df.values.tolist()])                  
                                 
                            ])


def pos_neg_neutral(col):
    if col>=POS_NEG_NEUT:
        #positive
        return 1
    elif col<= -POS_NEG_NEUT:
        #negative
        return -1
    
    else:
        return 0
    
    
    
@app.callback(Output('recent-tweet-table'))

                                 
                                 ]))
                         
                     ])
    

                               
        
        






SyntaxError: invalid syntax (<ipython-input-2-a81f11e853d0>, line 149)